In [1]:
import pandas as pd
import itertools as itt
import numbers as nb
import numpy as np
import re
import math
import statistics as st
import copy

from cleaning_utils import pivot_dataset, transform_chemical_data

In [2]:
idaho_data = pd.read_csv("Datasets/idaho_data.csv")

/tmp/ipykernel_237011/1784059464.py:1: DtypeWarning: Columns (13,14,17,18,19,21,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  idaho_data = pd.read_csv("Datasets/idaho_data.csv")


In [3]:
desired_chemical_names = ["Chloride", "Sulfate", "Hardness", "Sodium", "Potassium", "Magnesium", "Calcium",
                          "Specific Conductance", "Total Dissolved Solids", "Water Temperature", "pH"]

dataset_pivoted = pivot_dataset(
    idaho_data,
    sample_id_columns=["SampleNumber", "SampleDate"],
    per_sample_data=["Latitude", "Longitude"],
    chemical_name_column="CharName",
    values_per_chemical=["Amount", "UOM", "MinDetectLimit"],
    desired_chemical_names=desired_chemical_names,
    # drop_duplicates=True
)

dataset_pivoted

SampleNumber        SampleDate   Latitude   Longitude Calcium  \
                                                                      Amount   
0              0003ETAN655B  29/03/2000 10:20  43.847830 -112.709326  [65.6]   
1              0003ETAN655C  29/03/2000 10:20  43.847830 -112.709326     NaN   
2              0004ENRFIWDB  20/04/2000 10:05  43.651806 -112.919948   [896]   
3              0004ENRFIWDD  20/04/2000 10:05  43.651806 -112.919948     NaN   
4              0004ENRFSEWB   20/04/2000 9:40  43.653774 -112.912298   [8.4]   
...                     ...               ...        ...         ...     ...   
28690  IDEQ-2017-11-08-6159   8/11/2017 14:32  43.376230 -116.550910     NaN   
28691  IDEQ-2017-11-14-6160  14/11/2017 10:55  43.717750 -116.998090     NaN   
28692  IDEQ-2017-11-14-6161  14/11/2017 11:40  43.713400 -116.902030     NaN   
28693  IDEQ-2017-11-14-6162  14/11/2017 12:17  43.697140 -117.020360     NaN   
28694  IDEQ-2017-11-20-6163  20/11/2017 10:50  43.897710 -111.157300    [66]   

                             Chloride                         ... Sulfate  \
      MinDetectLimit     UOM   Amount MinDetectLimit     UOM  ...     UOM   
0              [0.1]  [mg/l]      NaN            NaN     NaN  ...     NaN   
1                NaN     NaN    [411]          [0.9]  [mg/l]  ...  [mg/l]   
2              [0.1]  [mg/l]      NaN            NaN     NaN  ...     NaN   
3                NaN     NaN   [9713]          [0.9]  [mg/l]  ...  [mg/l]   
4              [0.1]  [mg/l]      NaN            NaN     NaN  ...     NaN   
...              ...     ...      ...            ...     ...  ...     ...   
28690            NaN     NaN      NaN            NaN     NaN  ...     NaN   
28691            NaN     NaN      NaN            NaN     NaN  ...     NaN   
28692            NaN     NaN      NaN            NaN     NaN  ...     NaN   
28693            NaN     NaN      NaN            NaN     NaN  ...     NaN   
28694          [nan]  [mg/l]   [32.3]          [nan]  [mg/l]  ...  [mg/l]   

      Total Dissolved Solids                        Water Temperature  \
                      Amount MinDetectLimit     UOM            Amount   
0                        NaN            NaN     NaN               NaN   
1                     [1320]          [1.0]  [mg/l]               NaN   
2                        NaN            NaN     NaN               NaN   
3                    [17100]          [1.0]  [mg/l]               NaN   
4                        NaN            NaN     NaN               NaN   
...                      ...            ...     ...               ...   
28690                    NaN            NaN     NaN           [15.79]   
28691                    NaN            NaN     NaN           [18.94]   
28692                    NaN            NaN     NaN           [15.42]   
28693                    NaN            NaN     NaN           [16.51]   
28694                    NaN            NaN     NaN            [9.02]   

                                pH                       
      MinDetectLimit   UOM  Amount MinDetectLimit   UOM  
0                NaN   NaN     NaN            NaN   NaN  
1                NaN   NaN     NaN            NaN   NaN  
2                NaN   NaN     NaN            NaN   NaN  
3                NaN   NaN     NaN            NaN   NaN  
4                NaN   NaN     NaN            NaN   NaN  
...              ...   ...     ...            ...   ...  
28690          [nan]  [°C]  [7.56]          [nan]  [pH]  
28691          [nan]  [°C]  [7.14]          [nan]  [pH]  
28692          [nan]  [°C]  [6.95]          [nan]  [pH]  
28693          [nan]  [°C]  [7.42]          [nan]  [pH]  
28694          [nan]  [°C]   [6.9]          [nan]  [pH]  

[28695 rows x 37 columns]

In [4]:
# Part 2 apply formatting to each cell and expand

In [5]:
# remove units whitespace and convert to lowercase

def clean_units(units):
    if isinstance(units, list):
        return ["".join(unit.strip().lower().split()) if isinstance(unit, str) else unit for unit in units]
    elif isinstance(units, str):
        return "".join(units.strip().lower().split())
    else:
        return np.nan


units_index = list(itt.product(desired_chemical_names, ["UOM"]))
dataset_pivoted[units_index] = dataset_pivoted[units_index].applymap(
    clean_units)

dataset_pivoted

SampleNumber        SampleDate   Latitude   Longitude Calcium  \
                                                                      Amount   
0              0003ETAN655B  29/03/2000 10:20  43.847830 -112.709326  [65.6]   
1              0003ETAN655C  29/03/2000 10:20  43.847830 -112.709326     NaN   
2              0004ENRFIWDB  20/04/2000 10:05  43.651806 -112.919948   [896]   
3              0004ENRFIWDD  20/04/2000 10:05  43.651806 -112.919948     NaN   
4              0004ENRFSEWB   20/04/2000 9:40  43.653774 -112.912298   [8.4]   
...                     ...               ...        ...         ...     ...   
28690  IDEQ-2017-11-08-6159   8/11/2017 14:32  43.376230 -116.550910     NaN   
28691  IDEQ-2017-11-14-6160  14/11/2017 10:55  43.717750 -116.998090     NaN   
28692  IDEQ-2017-11-14-6161  14/11/2017 11:40  43.713400 -116.902030     NaN   
28693  IDEQ-2017-11-14-6162  14/11/2017 12:17  43.697140 -117.020360     NaN   
28694  IDEQ-2017-11-20-6163  20/11/2017 10:50  43.897710 -111.157300    [66]   

                             Chloride                         ... Sulfate  \
      MinDetectLimit     UOM   Amount MinDetectLimit     UOM  ...     UOM   
0              [0.1]  [mg/l]      NaN            NaN     NaN  ...     NaN   
1                NaN     NaN    [411]          [0.9]  [mg/l]  ...  [mg/l]   
2              [0.1]  [mg/l]      NaN            NaN     NaN  ...     NaN   
3                NaN     NaN   [9713]          [0.9]  [mg/l]  ...  [mg/l]   
4              [0.1]  [mg/l]      NaN            NaN     NaN  ...     NaN   
...              ...     ...      ...            ...     ...  ...     ...   
28690            NaN     NaN      NaN            NaN     NaN  ...     NaN   
28691            NaN     NaN      NaN            NaN     NaN  ...     NaN   
28692            NaN     NaN      NaN            NaN     NaN  ...     NaN   
28693            NaN     NaN      NaN            NaN     NaN  ...     NaN   
28694          [nan]  [mg/l]   [32.3]          [nan]  [mg/l]  ...  [mg/l]   

      Total Dissolved Solids                        Water Temperature  \
                      Amount MinDetectLimit     UOM            Amount   
0                        NaN            NaN     NaN               NaN   
1                     [1320]          [1.0]  [mg/l]               NaN   
2                        NaN            NaN     NaN               NaN   
3                    [17100]          [1.0]  [mg/l]               NaN   
4                        NaN            NaN     NaN               NaN   
...                      ...            ...     ...               ...   
28690                    NaN            NaN     NaN           [15.79]   
28691                    NaN            NaN     NaN           [18.94]   
28692                    NaN            NaN     NaN           [15.42]   
28693                    NaN            NaN     NaN           [16.51]   
28694                    NaN            NaN     NaN            [9.02]   

                                pH                       
      MinDetectLimit   UOM  Amount MinDetectLimit   UOM  
0                NaN   NaN     NaN            NaN   NaN  
1                NaN   NaN     NaN            NaN   NaN  
2                NaN   NaN     NaN            NaN   NaN  
3                NaN   NaN     NaN            NaN   NaN  
4                NaN   NaN     NaN            NaN   NaN  
...              ...   ...     ...            ...   ...  
28690          [nan]  [°c]  [7.56]          [nan]  [ph]  
28691          [nan]  [°c]  [7.14]          [nan]  [ph]  
28692          [nan]  [°c]  [6.95]          [nan]  [ph]  
28693          [nan]  [°c]  [7.42]          [nan]  [ph]  
28694          [nan]  [°c]   [6.9]          [nan]  [ph]  

[28695 rows x 37 columns]

In [6]:
# This function should format a single amount value and output a float-prefix tuple.
def format_amount(erase_invalid: bool = False):

    def format_amount_func(amount, min_detection_limit, uom):

        # if amount is already number
        if isinstance(amount, nb.Number):
            return ('=', float(amount), uom)

        if isinstance(amount, str):
            # remove whitespace
            amount = ''.join(amount.split())
            amount_prefix = None

            # if amount matches prefix-numerical form
            if re.fullmatch(r'([<>=]?)[0-9]*(\.?)[0-9]+', amount):
                if re.match(r'[<>=]', amount):
                    amount_prefix = re.split(r'[<>=]', amount)
                    amount_prefix[1] = float(amount_prefix[1])
                    amount_prefix = (amount_prefix[0], amount_prefix[1], uom)
                else:
                    amount_prefix = ("=", float(amount), uom)
                return amount_prefix

            # if amount is below detection limit
            elif amount == "BDL" or amount == "ND":
                if min_detection_limit is not None:
                    return ("<", min_detection_limit, uom)
                else:
                    return ("=", 0, uom)

        # otherwise amount is invalid
        if erase_invalid:
            return (np.nan, np.nan, np.nan)
        else:
            raise ValueError("Invalid Formatting - " + str(amount))

    return format_amount_func


# apply formatting to every chemical and sort columns
for chemical_name in desired_chemical_names:
    transform_chemical_data(dataset_pivoted, chemical_name, format_amount(True),
                            ["Amount", "MinDetectLimit", "UOM"], ["Prefix", "Amount", "UOM"], split_lists=True)

dataset_pivoted = dataset_pivoted.sort_index(axis=1)
dataset_pivoted

Calcium                                Chloride                        \
        Amount MinDetectLimit Prefix     UOM    Amount MinDetectLimit Prefix   
0       [65.6]          [0.1]    [=]  [mg/l]       NaN            NaN      =   
1          NaN            NaN      =     NaN   [411.0]          [0.9]    [=]   
2      [896.0]          [0.1]    [=]  [mg/l]       NaN            NaN      =   
3          NaN            NaN      =     NaN  [9713.0]          [0.9]    [=]   
4        [8.4]          [0.1]    [=]  [mg/l]       NaN            NaN      =   
...        ...            ...    ...     ...       ...            ...    ...   
28690      NaN            NaN      =     NaN       NaN            NaN      =   
28691      NaN            NaN      =     NaN       NaN            NaN      =   
28692      NaN            NaN      =     NaN       NaN            NaN      =   
28693      NaN            NaN      =     NaN       NaN            NaN      =   
28694   [66.0]          [nan]    [=]  [mg/l]    [32.3]          [nan]    [=]   

              Hardness                 ... Total Dissolved Solids          \
          UOM   Amount MinDetectLimit  ...                 Prefix     UOM   
0         NaN      NaN            NaN  ...                      =     NaN   
1      [mg/l]      NaN            NaN  ...                    [=]  [mg/l]   
2         NaN      NaN            NaN  ...                      =     NaN   
3      [mg/l]      NaN            NaN  ...                    [=]  [mg/l]   
4         NaN      NaN            NaN  ...                      =     NaN   
...       ...      ...            ...  ...                    ...     ...   
28690     NaN      NaN            NaN  ...                      =     NaN   
28691     NaN      NaN            NaN  ...                      =     NaN   
28692     NaN      NaN            NaN  ...                      =     NaN   
28693     NaN      NaN            NaN  ...                      =     NaN   
28694  [mg/l]      NaN            NaN  ...                      =     NaN   

      Water Temperature                                  pH                 \
                 Amount MinDetectLimit Prefix   UOM  Amount MinDetectLimit   
0                   NaN            NaN      =   NaN     NaN            NaN   
1                   NaN            NaN      =   NaN     NaN            NaN   
2                   NaN            NaN      =   NaN     NaN            NaN   
3                   NaN            NaN      =   NaN     NaN            NaN   
4                   NaN            NaN      =   NaN     NaN            NaN   
...                 ...            ...    ...   ...     ...            ...   
28690           [15.79]          [nan]    [=]  [°c]  [7.56]          [nan]   
28691           [18.94]          [nan]    [=]  [°c]  [7.14]          [nan]   
28692           [15.42]          [nan]    [=]  [°c]  [6.95]          [nan]   
28693           [16.51]          [nan]    [=]  [°c]  [7.42]          [nan]   
28694            [9.02]          [nan]    [=]  [°c]   [6.9]          [nan]   

                    
      Prefix   UOM  
0          =   NaN  
1          =   NaN  
2          =   NaN  
3          =   NaN  
4          =   NaN  
...      ...   ...  
28690    [=]  [ph]  
28691    [=]  [ph]  
28692    [=]  [ph]  
28693    [=]  [ph]  
28694    [=]  [ph]  

[28695 rows x 48 columns]

In [7]:

# standardise units

def standardise_units(units_dict):
    def standardise_units_func(amount, min_detection_limit, prefix, uom):
        if isinstance(uom, float):
            if math.isnan(uom):
                return (np.nan, np.nan, np.nan, np.nan)
        if not (uom in units_dict.keys()):
            raise ValueError('Invalid units', uom)
        amount = units_dict[uom](amount)
        min_detection_limit = units_dict[uom](min_detection_limit)
        return (amount, min_detection_limit, prefix, uom)
    return standardise_units_func


convert_to_standard = dict()
get_standard = dict()

# mass/volume concentration g/L
convert_to_standard['g/l'] = lambda x: x
convert_to_standard['mg/l'] = lambda x: x/1e3
convert_to_standard['ug/l'] = lambda x: x/1e6
convert_to_standard['µg/l'] = lambda x: x/1e6
convert_to_standard['ng/l'] = lambda x: x/1e9

# pH
convert_to_standard['ph'] = lambda x: x

# temperature °c
convert_to_standard['°c'] = lambda x: x
convert_to_standard['°f'] = lambda x: 5*(x-32)/9

# conductivity us/cm
convert_to_standard['us/cm'] = lambda x: x
convert_to_standard['µs/cm'] = lambda x: x


# apply conversion
for chemical_name in desired_chemical_names:
    transform_chemical_data(dataset_pivoted, chemical_name, standardise_units(convert_to_standard),
                            ["Amount", "MinDetectLimit", "Prefix", "UOM"], ["Amount", "MinDetectLimit", "Prefix", "UOM"], split_lists=True)
    dataset_pivoted = dataset_pivoted.sort_index(axis=1)


dataset_pivoted

Calcium                                Chloride                        \
        Amount MinDetectLimit Prefix     UOM    Amount MinDetectLimit Prefix   
0       [65.6]          [0.1]    [=]  [mg/l]       NaN            NaN      =   
1          NaN            NaN      =     NaN   [411.0]          [0.9]    [=]   
2      [896.0]          [0.1]    [=]  [mg/l]       NaN            NaN      =   
3          NaN            NaN      =     NaN  [9713.0]          [0.9]    [=]   
4        [8.4]          [0.1]    [=]  [mg/l]       NaN            NaN      =   
...        ...            ...    ...     ...       ...            ...    ...   
28690      NaN            NaN      =     NaN       NaN            NaN      =   
28691      NaN            NaN      =     NaN       NaN            NaN      =   
28692      NaN            NaN      =     NaN       NaN            NaN      =   
28693      NaN            NaN      =     NaN       NaN            NaN      =   
28694   [66.0]          [nan]    [=]  [mg/l]    [32.3]          [nan]    [=]   

              Hardness                 ... Total Dissolved Solids          \
          UOM   Amount MinDetectLimit  ...                 Prefix     UOM   
0         NaN      NaN            NaN  ...                      =     NaN   
1      [mg/l]      NaN            NaN  ...                    [=]  [mg/l]   
2         NaN      NaN            NaN  ...                      =     NaN   
3      [mg/l]      NaN            NaN  ...                    [=]  [mg/l]   
4         NaN      NaN            NaN  ...                      =     NaN   
...       ...      ...            ...  ...                    ...     ...   
28690     NaN      NaN            NaN  ...                      =     NaN   
28691     NaN      NaN            NaN  ...                      =     NaN   
28692     NaN      NaN            NaN  ...                      =     NaN   
28693     NaN      NaN            NaN  ...                      =     NaN   
28694  [mg/l]      NaN            NaN  ...                      =     NaN   

      Water Temperature                                  pH                 \
                 Amount MinDetectLimit Prefix   UOM  Amount MinDetectLimit   
0                   NaN            NaN      =   NaN     NaN            NaN   
1                   NaN            NaN      =   NaN     NaN            NaN   
2                   NaN            NaN      =   NaN     NaN            NaN   
3                   NaN            NaN      =   NaN     NaN            NaN   
4                   NaN            NaN      =   NaN     NaN            NaN   
...                 ...            ...    ...   ...     ...            ...   
28690           [15.79]          [nan]    [=]  [°c]  [7.56]          [nan]   
28691           [18.94]          [nan]    [=]  [°c]  [7.14]          [nan]   
28692           [15.42]          [nan]    [=]  [°c]  [6.95]          [nan]   
28693           [16.51]          [nan]    [=]  [°c]  [7.42]          [nan]   
28694            [9.02]          [nan]    [=]  [°c]   [6.9]          [nan]   

                    
      Prefix   UOM  
0        NaN   NaN  
1        NaN   NaN  
2        NaN   NaN  
3        NaN   NaN  
4        NaN   NaN  
...      ...   ...  
28690    [=]  [ph]  
28691    [=]  [ph]  
28692    [=]  [ph]  
28693    [=]  [ph]  
28694    [=]  [ph]  

[28695 rows x 48 columns]

In [8]:
# drop units and min detection limit

dataset_formatted = copy.deepcopy(dataset_pivoted)

units_min_index = list(itt.product(
    desired_chemical_names, ["UOM", "MinDetectLimit"]))
dataset_formatted = dataset_formatted.drop(units_min_index, axis=1)

# move chemical columns to left
other_params = set(dataset_formatted.columns.levels[0])
desired_chemical_names.sort()
other_params = list(other_params.difference(set(desired_chemical_names)))
ordered_params = other_params
ordered_params.extend(desired_chemical_names)
new_cols = dataset_formatted.columns.reindex(ordered_params, level=0)
dataset_formatted = dataset_formatted.reindex(columns=new_cols[0])

dataset_formatted

SampleDate   Longitude          SampleNumber   Latitude  Calcium  \
                                                                       Amount   
0      29/03/2000 10:20 -112.709326          0003ETAN655B  43.847830   [65.6]   
1      29/03/2000 10:20 -112.709326          0003ETAN655C  43.847830      NaN   
2      20/04/2000 10:05 -112.919948          0004ENRFIWDB  43.651806  [896.0]   
3      20/04/2000 10:05 -112.919948          0004ENRFIWDD  43.651806      NaN   
4       20/04/2000 9:40 -112.912298          0004ENRFSEWB  43.653774    [8.4]   
...                 ...         ...                   ...        ...      ...   
28690   8/11/2017 14:32 -116.550910  IDEQ-2017-11-08-6159  43.376230      NaN   
28691  14/11/2017 10:55 -116.998090  IDEQ-2017-11-14-6160  43.717750      NaN   
28692  14/11/2017 11:40 -116.902030  IDEQ-2017-11-14-6161  43.713400      NaN   
28693  14/11/2017 12:17 -117.020360  IDEQ-2017-11-14-6162  43.697140      NaN   
28694  20/11/2017 10:50 -111.157300  IDEQ-2017-11-20-6163  43.897710   [66.0]   

              Chloride        Hardness         ... Specific Conductance  \
      Prefix    Amount Prefix   Amount Prefix  ...               Amount   
0        [=]       NaN      =      NaN      =  ...                  NaN   
1          =   [411.0]    [=]      NaN      =  ...                  NaN   
2        [=]       NaN      =      NaN      =  ...                  NaN   
3          =  [9713.0]    [=]      NaN      =  ...                  NaN   
4        [=]       NaN      =      NaN      =  ...                  NaN   
...      ...       ...    ...      ...    ...  ...                  ...   
28690      =       NaN      =      NaN      =  ...              [817.0]   
28691      =       NaN      =      NaN      =  ...              [494.0]   
28692      =       NaN      =      NaN      =  ...             [1270.0]   
28693      =       NaN      =      NaN      =  ...              [415.0]   
28694    [=]    [32.3]    [=]      NaN      =  ...              [699.0]   

              Sulfate        Total Dissolved Solids        Water Temperature  \
      Prefix   Amount Prefix                 Amount Prefix            Amount   
0          =      NaN      =                    NaN      =               NaN   
1          =  [302.0]    [=]               [1320.0]    [=]               NaN   
2          =      NaN      =                    NaN      =               NaN   
3          =   [24.6]    [=]              [17100.0]    [=]               NaN   
4          =      NaN      =                    NaN      =               NaN   
...      ...      ...    ...                    ...    ...               ...   
28690    [=]      NaN      =                    NaN      =           [15.79]   
28691    [=]      NaN      =                    NaN      =           [18.94]   
28692    [=]      NaN      =                    NaN      =           [15.42]   
28693    [=]      NaN      =                    NaN      =           [16.51]   
28694    [=]   [32.4]    [=]                    NaN      =            [9.02]   

                  pH         
      Prefix  Amount Prefix  
0          =     NaN    NaN  
1          =     NaN    NaN  
2          =     NaN    NaN  
3          =     NaN    NaN  
4          =     NaN    NaN  
...      ...     ...    ...  
28690    [=]  [7.56]    [=]  
28691    [=]  [7.14]    [=]  
28692    [=]  [6.95]    [=]  
28693    [=]  [7.42]    [=]  
28694    [=]   [6.9]    [=]  

[28695 rows x 26 columns]

In [9]:
# turn list of measurements into one aggregated measurement

def agg_measurement(amount, prefix):
    if not isinstance(amount, list):
        if math.isnan(amount):
            return (np.nan, np.nan)
        elif isinstance(amount, nb.Number):
            if isinstance(prefix, list):
                raise TypeError(prefix)
            return (amount, prefix)
        else:
            raise TypeError(amount)

    if set(prefix) == set(["<"]):
        return (min(amount), "<")
    else:
        return (st.mean(amount), "=")


for chemical_name in desired_chemical_names:
    transform_chemical_data(dataset_formatted, chemical_name, agg_measurement,
                            ["Amount", "Prefix"], ["Amount", "Prefix"])

dataset_formatted

SampleDate   Longitude          SampleNumber   Latitude Calcium  \
                                                                      Amount   
0      29/03/2000 10:20 -112.709326          0003ETAN655B  43.847830    65.6   
1      29/03/2000 10:20 -112.709326          0003ETAN655C  43.847830     NaN   
2      20/04/2000 10:05 -112.919948          0004ENRFIWDB  43.651806   896.0   
3      20/04/2000 10:05 -112.919948          0004ENRFIWDD  43.651806     NaN   
4       20/04/2000 9:40 -112.912298          0004ENRFSEWB  43.653774     8.4   
...                 ...         ...                   ...        ...     ...   
28690   8/11/2017 14:32 -116.550910  IDEQ-2017-11-08-6159  43.376230     NaN   
28691  14/11/2017 10:55 -116.998090  IDEQ-2017-11-14-6160  43.717750     NaN   
28692  14/11/2017 11:40 -116.902030  IDEQ-2017-11-14-6161  43.713400     NaN   
28693  14/11/2017 12:17 -117.020360  IDEQ-2017-11-14-6162  43.697140     NaN   
28694  20/11/2017 10:50 -111.157300  IDEQ-2017-11-20-6163  43.897710    66.0   

             Chloride        Hardness         ... Specific Conductance         \
      Prefix   Amount Prefix   Amount Prefix  ...               Amount Prefix   
0          =      NaN    NaN      NaN    NaN  ...                  NaN    NaN   
1        NaN    411.0      =      NaN    NaN  ...                  NaN    NaN   
2          =      NaN    NaN      NaN    NaN  ...                  NaN    NaN   
3        NaN   9713.0      =      NaN    NaN  ...                  NaN    NaN   
4          =      NaN    NaN      NaN    NaN  ...                  NaN    NaN   
...      ...      ...    ...      ...    ...  ...                  ...    ...   
28690    NaN      NaN    NaN      NaN    NaN  ...                817.0      =   
28691    NaN      NaN    NaN      NaN    NaN  ...                494.0      =   
28692    NaN      NaN    NaN      NaN    NaN  ...               1270.0      =   
28693    NaN      NaN    NaN      NaN    NaN  ...                415.0      =   
28694      =     32.3      =      NaN    NaN  ...                699.0      =   

      Sulfate        Total Dissolved Solids        Water Temperature         \
       Amount Prefix                 Amount Prefix            Amount Prefix   
0         NaN    NaN                    NaN    NaN               NaN    NaN   
1       302.0      =                 1320.0      =               NaN    NaN   
2         NaN    NaN                    NaN    NaN               NaN    NaN   
3        24.6      =                17100.0      =               NaN    NaN   
4         NaN    NaN                    NaN    NaN               NaN    NaN   
...       ...    ...                    ...    ...               ...    ...   
28690     NaN    NaN                    NaN    NaN             15.79      =   
28691     NaN    NaN                    NaN    NaN             18.94      =   
28692     NaN    NaN                    NaN    NaN             15.42      =   
28693     NaN    NaN                    NaN    NaN             16.51      =   
28694    32.4      =                    NaN    NaN              9.02      =   

          pH         
      Amount Prefix  
0        NaN    NaN  
1        NaN    NaN  
2        NaN    NaN  
3        NaN    NaN  
4        NaN    NaN  
...      ...    ...  
28690   7.56      =  
28691   7.14      =  
28692   6.95      =  
28693   7.42      =  
28694   6.90      =  

[28695 rows x 26 columns]

In [10]:
# remove samples with too many NaN values

dataset_formatted2 = copy.deepcopy(dataset_formatted)


def count_nas(row):
    return row.isna().sum()


na_threshold = 5
amounts_index = list(itt.product(desired_chemical_names, ["Amount"]))
mask = dataset_formatted2[amounts_index].apply(count_nas, axis=1)
mask = mask[mask > na_threshold].index

dataset_formatted2 = dataset_formatted2.loc[mask]
dataset_formatted2 = dataset_formatted2.reset_index()
dataset_formatted2 = dataset_formatted2.drop(['index'], axis=1)

dataset_formatted2

/tmp/ipykernel_237011/1751125993.py:17: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  dataset_formatted2 = dataset_formatted2.drop(['index'], axis=1)


SampleDate   Longitude          SampleNumber   Latitude Calcium  \
                                                                      Amount   
0      29/03/2000 10:20 -112.709326          0003ETAN655B  43.847830    65.6   
1      29/03/2000 10:20 -112.709326          0003ETAN655C  43.847830     NaN   
2      20/04/2000 10:05 -112.919948          0004ENRFIWDB  43.651806   896.0   
3      20/04/2000 10:05 -112.919948          0004ENRFIWDD  43.651806     NaN   
4       20/04/2000 9:40 -112.912298          0004ENRFSEWB  43.653774     8.4   
...                 ...         ...                   ...        ...     ...   
16470   8/11/2017 11:41 -116.989560  IDEQ-2017-11-08-6157  43.704780     NaN   
16471   8/11/2017 14:32 -116.550910  IDEQ-2017-11-08-6159  43.376230     NaN   
16472  14/11/2017 10:55 -116.998090  IDEQ-2017-11-14-6160  43.717750     NaN   
16473  14/11/2017 11:40 -116.902030  IDEQ-2017-11-14-6161  43.713400     NaN   
16474  14/11/2017 12:17 -117.020360  IDEQ-2017-11-14-6162  43.697140     NaN   

             Chloride        Hardness         ... Specific Conductance         \
      Prefix   Amount Prefix   Amount Prefix  ...               Amount Prefix   
0          =      NaN    NaN      NaN    NaN  ...                  NaN    NaN   
1        NaN    411.0      =      NaN    NaN  ...                  NaN    NaN   
2          =      NaN    NaN      NaN    NaN  ...                  NaN    NaN   
3        NaN   9713.0      =      NaN    NaN  ...                  NaN    NaN   
4          =      NaN    NaN      NaN    NaN  ...                  NaN    NaN   
...      ...      ...    ...      ...    ...  ...                  ...    ...   
16470    NaN      NaN    NaN      NaN    NaN  ...               1300.0      =   
16471    NaN      NaN    NaN      NaN    NaN  ...                817.0      =   
16472    NaN      NaN    NaN      NaN    NaN  ...                494.0      =   
16473    NaN      NaN    NaN      NaN    NaN  ...               1270.0      =   
16474    NaN      NaN    NaN      NaN    NaN  ...                415.0      =   

      Sulfate        Total Dissolved Solids        Water Temperature         \
       Amount Prefix                 Amount Prefix            Amount Prefix   
0         NaN    NaN                    NaN    NaN               NaN    NaN   
1       302.0      =                 1320.0      =               NaN    NaN   
2         NaN    NaN                    NaN    NaN               NaN    NaN   
3        24.6      =                17100.0      =               NaN    NaN   
4         NaN    NaN                    NaN    NaN               NaN    NaN   
...       ...    ...                    ...    ...               ...    ...   
16470     NaN    NaN                    NaN    NaN             14.55      =   
16471     NaN    NaN                    NaN    NaN             15.79      =   
16472     NaN    NaN                    NaN    NaN             18.94      =   
16473     NaN    NaN                    NaN    NaN             15.42      =   
16474     NaN    NaN                    NaN    NaN             16.51      =   

          pH         
      Amount Prefix  
0        NaN    NaN  
1        NaN    NaN  
2        NaN    NaN  
3        NaN    NaN  
4        NaN    NaN  
...      ...    ...  
16470   7.25      =  
16471   7.56      =  
16472   7.14      =  
16473   6.95      =  
16474   7.42      =  

[16475 rows x 26 columns]

In [11]:
# calculate averages for each chemical
# for now I naively take the mean regardless of < or =.
amount_avgs = dict()
missing_chemicals = []

for chemical_name in desired_chemical_names:
    amount_avgs[chemical_name] = dataset_formatted2[chemical_name,
                                                    "Amount"].mean()
    if math.isnan(amount_avgs[chemical_name]):
        missing_chemicals.append(chemical_name)

# remove chemicals with all NaNs
desired_chemical_names = list(
    set(desired_chemical_names).difference(set(missing_chemicals)))
dataset_formatted2 = dataset_formatted2.drop(missing_chemicals, axis=1)

# fill NaN values with averages
for chemical_name in desired_chemical_names:
    transform_chemical_data(dataset_formatted2, chemical_name, lambda a, p: (amount_avgs[chemical_name], '=') if math.isnan(a) else (a, p),
                            ["Amount", "Prefix"], ["Amount", "Prefix"])

In [12]:
dataset_formatted2

SampleDate   Longitude          SampleNumber   Latitude  \
                                                                       
0      29/03/2000 10:20 -112.709326          0003ETAN655B  43.847830   
1      29/03/2000 10:20 -112.709326          0003ETAN655C  43.847830   
2      20/04/2000 10:05 -112.919948          0004ENRFIWDB  43.651806   
3      20/04/2000 10:05 -112.919948          0004ENRFIWDD  43.651806   
4       20/04/2000 9:40 -112.912298          0004ENRFSEWB  43.653774   
...                 ...         ...                   ...        ...   
16470   8/11/2017 11:41 -116.989560  IDEQ-2017-11-08-6157  43.704780   
16471   8/11/2017 14:32 -116.550910  IDEQ-2017-11-08-6159  43.376230   
16472  14/11/2017 10:55 -116.998090  IDEQ-2017-11-14-6160  43.717750   
16473  14/11/2017 11:40 -116.902030  IDEQ-2017-11-14-6161  43.713400   
16474  14/11/2017 12:17 -117.020360  IDEQ-2017-11-14-6162  43.697140   

          Calcium            Chloride          Magnesium         ...  \
           Amount Prefix       Amount Prefix      Amount Prefix  ...   
0       65.600000      =    54.634687      =   19.800000      =  ...   
1       65.254819      =   411.000000      =   21.731828      =  ...   
2      896.000000      =    54.634687      =  289.000000      =  ...   
3       65.254819      =  9713.000000      =   21.731828      =  ...   
4        8.400000      =    54.634687      =    2.000000      =  ...   
...           ...    ...          ...    ...         ...    ...  ...   
16470   65.254819      =    54.634687      =   21.731828      =  ...   
16471   65.254819      =    54.634687      =   21.731828      =  ...   
16472   65.254819      =    54.634687      =   21.731828      =  ...   
16473   65.254819      =    54.634687      =   21.731828      =  ...   
16474   65.254819      =    54.634687      =   21.731828      =  ...   

      Specific Conductance            Sulfate        Total Dissolved Solids  \
                    Amount Prefix      Amount Prefix                 Amount   
0               712.114405      =   89.282411      =             704.492179   
1               712.114405      =  302.000000      =            1320.000000   
2               712.114405      =   89.282411      =             704.492179   
3               712.114405      =   24.600000      =           17100.000000   
4               712.114405      =   89.282411      =             704.492179   
...                    ...    ...         ...    ...                    ...   
16470          1300.000000      =   89.282411      =             704.492179   
16471           817.000000      =   89.282411      =             704.492179   
16472           494.000000      =   89.282411      =             704.492179   
16473          1270.000000      =   89.282411      =             704.492179   
16474           415.000000      =   89.282411      =             704.492179   

             Water Temperature               pH         
      Prefix            Amount Prefix    Amount Prefix  
0          =         14.455916      =  8.344642      =  
1          =         14.455916      =  8.344642      =  
2          =         14.455916      =  8.344642      =  
3          =         14.455916      =  8.344642      =  
4          =         14.455916      =  8.344642      =  
...      ...               ...    ...       ...    ...  
16470      =         14.550000      =  7.250000      =  
16471      =         15.790000      =  7.560000      =  
16472      =         18.940000      =  7.140000      =  
16473      =         15.420000      =  6.950000      =  
16474      =         16.510000      =  7.420000      =  

[16475 rows x 24 columns]